#### Current issues: 
- Vehicles used after sold (fixing)
- Carbon emission calculated wrong (fixing)
- All vehicles are diesel, not sure if carbon emission cap is satisfied (please help me to check)
- Ending dataframe not ready (please use my current result to set up a dataframe structure)
- Current cost not calculated (my termial can tell me a result, but it will still be nice if Victor's excel can explain specifics)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp as pl
pd.set_option('display.max_rows', 200000)

In [2]:
carbon_emissions_df  = pd.read_csv('carbon_emissions.csv')
resale_df = pd.read_csv('cost_profiles.csv')
demand_df  = pd.read_csv('demand.csv')
fuels_df = pd.read_csv('fuels.csv')
vehicles_df  = pd.read_csv('vehicles.csv')
vehicles_fuels_df  = pd.read_csv('vehicles_fuels.csv')
sample_submission_df = pd.read_csv('sample_submission.csv') 

### Edit vehicles_df to include all distance bucket selections

In [3]:
# Define a function to generate additional rows based on 'Distance'
def generate_rows(row):
    if row['Distance'] == 'D4':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D3'}]),
            pd.DataFrame([{**row, 'Distance': 'D2'}]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    elif row['Distance'] == 'D3':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D2'}]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    elif row['Distance'] == 'D2':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    else:
        return pd.DataFrame([row])

# Apply the function to each row and concatenate the results
vehicles_df_add = pd.concat(vehicles_df.apply(generate_rows, axis=1).tolist(), ignore_index=True)

### Initialize the Pulp problem and define decision variables

Unfortunatly, we will assume the consumption will be maximised for selected vehicle for now.

In [4]:
# Initialize the PuLP problem
prob = pl.LpProblem("Fleet_Transition_Optimization", pl.LpMinimize)

# Define Fuel type
fuel_type=['Electricity', 'B20', 'LNG', 'BioLNG', 'HVO']

# Define Distance Bucket type
distance_bucket = ['D1', 'D2', 'D3', 'D4']

# Define decision variables
buy_vars = pl.LpVariable.dicts("Buy", 
                               ((year, vehicle_id) for year in range(2023, 2039) for vehicle_id in vehicles_df['ID']),
                               lowBound=0, cat='Integer')

use_vars = pl.LpVariable.dicts("Use", 
                               ((year, vehicle_id, fuel, db) for year in range(2023, 2039) for vehicle_id in vehicles_df_add['ID']
                               for fuel in fuel_type for db in distance_bucket),
                               lowBound=0, cat='Integer')

sell_vars = pl.LpVariable.dicts("Sell", 
                                ((year, vehicle_id) for year in range(2023, 2039) for vehicle_id in vehicles_df['ID']),
                                lowBound=0, cat='Integer')

### Define objective function

The total cost consists of three components:
- Buying Cost for the vehicle
- Using Cost for the vehicle
- (Negative/Gain) Resale cost

Where the Using cost consists of three components:
- Fuel cost
- Insurance Cost
- Maintenance Cost

The Fuel cost is calculated by:

`D_v_f`: Distance travelled by vehicle `v` using fuel `f`. **(Assumed max)**
Unit: km

**multiplies by**

`N_v_f`: Number of vehicles of type `v` driving fuel type `f`.

**multiplies by**

`m_v_f`: Fuel consumption of vehicle of type `v` driving with fuel type `f`.
Unit: unit_fuel/km

**multiplies by**

`C_f_yr`: Cost of unit fuel of type `f` in the year `yr`.
Unit: $/unit_fuel

In [5]:
# First calculate buying costs
total_cost = pl.lpSum([
    # Buying cost
    buy_vars[(year, row['ID'])] * row['Cost ($)']
    for year in range(2023, 2039)
    for _, row in vehicles_df.iterrows()
])

In [6]:
# Add insurance and maintenance costs
for year in range(2023, 2039):
    for v in vehicles_df['ID']:
        purchase_year = int(v.split('_')[-1])
        if year >= purchase_year:
            age = year - purchase_year+1
            if age < 10:
                # Normal vehicle price
                purchase_cost = vehicles_df.loc[vehicles_df['ID'] == v, 'Cost ($)'].values[0]
                
                # Insurance percentage by age
                insurance_cost_pct = resale_df.loc[resale_df['End of Year'] == age, 'Insurance Cost %'].values[0] / 100
                
                # Maintenance percentage by age
                maintenance_cost_pct = resale_df.loc[resale_df['End of Year'] == age, 'Maintenance Cost %'].values[0] / 100
                
                # Insurance cost 
                insurance_cost = purchase_cost * insurance_cost_pct
                
                # Maintenance cost
                maintenance_cost = purchase_cost * maintenance_cost_pct
                
                # Add insurance cost to total cost
                total_cost += insurance_cost * pl.lpSum([use_vars[(year, v, fuel, db)] for db in distance_bucket for fuel in fuel_type])
                
                # Add maintenance cost to total cost
                total_cost += maintenance_cost * pl.lpSum([use_vars[(year, v, fuel, db)] for db in distance_bucket for fuel in fuel_type])

Before we calculate, we will merge the dataframe vehicles_df, vehicles_fuels_df, and fuels_df.

First, merge fuels_df with vehicles_df_add using column `Year`

In [7]:
merged_vehicles_df = pd.merge(vehicles_fuels_df, vehicles_df_add, on='ID', how='outer')

In [8]:
merged_fuels_total_df = pd.merge(merged_vehicles_df, fuels_df, on=['Fuel', 'Year'], how = 'outer')

In [9]:
# Add fuel costs
for year in range(2023, 2039):
    for db in distance_bucket:
        for _, row in merged_fuels_total_df.iterrows():
            # Unique vehicle id
            vehicle_id = row['ID']

            # Max distance travelled by vehicle, unit: km
            distance = row['Yearly range (km)']

            # Fuel type
            fuel_t = row['Fuel']

            # Fuel cost, unit: $/unit
            fuel_cost = fuels_df[(fuels_df['Fuel'] == fuel_t) & (fuels_df['Year'] == year)]['Cost ($/unit_fuel)']

            # Fuel consumption, unit: unit/km
            fuel_consumption = row['Consumption (unit_fuel/km)']

            # Append calculated fuel cost to total cost
            ## Formula: Fuel_cost = number of vehicle v * Distance * Fule_Consumption * Fuel_cost
            total_cost += distance * fuel_cost * fuel_consumption * use_vars[(year, vehicle_id, fuel_t, db)]

In [10]:
# Add (Negative/Gain) Resale cost
for year in range(2023, 2039):
    for vehicle_id in vehicles_df['ID']:
        purchase_year = int(vehicle_id.split('_')[-1])
        if year > purchase_year:
            age = year - purchase_year + 1
            if age < 10:
                purchase_cost = merged_fuels_total_df.loc[merged_fuels_total_df['ID'] == vehicle_id, 'Cost ($)'].values[0]
                resale_value_pct = resale_df.loc[resale_df['End of Year'] == age, 'Resale Value %'].values[0] / 100
                resale_value = purchase_cost * resale_value_pct
                total_cost -= resale_value * sell_vars[(year, vehicle_id)]

In [11]:
prob += total_cost

***

### Setting Constraints

In [12]:
# 1. Ensure vehicles can only be bought in the specific year
for year in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        purchase_year = int(v.split('_')[-1])
        if year != purchase_year:
            prob += buy_vars[(year, v)] == 0

In [13]:
# 2. Ensure vehicle has a 10-year life and must be sold by the end of 10th year, and also must buy, then sell.
for v in merged_fuels_total_df['ID']:
    purchase_year = int(v.split('_')[-1])
    if purchase_year+10 <2040:
        prob += buy_vars[(purchase_year, v)] == pl.lpSum([sell_vars[(y, v)] for y in range(purchase_year, purchase_year+10)])
    else:
        prob += buy_vars[(purchase_year, v)] >= pl.lpSum([sell_vars[(y, v)] for y in range(purchase_year, 2039)])

In [14]:
# 3. Ensure vehicles are used only if they have been bought
for v in merged_fuels_total_df['ID']:
    purchase_year = int(v.split('_')[-1])

    prob += buy_vars[(purchase_year, v)] >= pl.lpSum([use_vars[(y, v, fuel, db)] for y in range(purchase_year, 2039) for fuel in fuel_type for db in distance_bucket])

    if purchase_year != 2023:
        for y in range(2023, purchase_year):
            for fuel in fuel_type:
                for db in distance_bucket:
                    prob += use_vars[(y, v, fuel, db)] == 0

In [15]:
# Ensure vehicles are used only if they haven't been sold
#for v in merged_fuels_total_df['ID']:
     

In [16]:
# 4. Every year at most 20% of the vehicles in the existing fleet can be sold
sell = pl.lpSum([sell_vars[2023, v] for v in vehicles_df['ID'] if int(v.split('_')[-1]) == 2023])
buy = pl.lpSum([buy_vars[2023, v] for v in vehicles_df['ID'] if int(v.split('_')[-1]) == 2023])
prob += sell<= 0.2 * buy

for year in range(2024, 2039):
    sell_temp = pl.lpSum([sell_vars[year, v] for v in vehicles_df['ID'] if int(v.split('_')[-1]) <= year])
    buy += pl.lpSum([buy_vars[year, v] for v in vehicles_df['ID'] if int(v.split('_')[-1]) == year])
    prob += sell_temp<= 0.2 * (buy-sell)
    sell += sell_temp

In [17]:
# 5. Ensure all vehicles can only use selected fuel type
for year in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        vt = v.split('_')[0]
        for ft in fuel_type:
            for db in distance_bucket:
                if ((vt == 'BEV') & (ft != 'Electricity')) | ((vt == 'Diesel') & (ft not in ['B20', 'HVO'])) | ((vt == 'LNG') & (ft not in ['LNG', 'BioLNG'])):
                    prob += use_vars[(year, v, ft, db)] == 0  

In [18]:
# 6. Ensure vehicle can't be used if not belongs to demand bucket defined in original dataframe
for year in range(2023, 2039):
    for db in ['D1', 'D2', 'D3', 'D4']:
        for v in vehicles_df['ID']:
            if merged_fuels_total_df[(merged_fuels_total_df['ID'] == v)& (merged_fuels_total_df['Distance'] == db)].empty:
                prob += use_vars[(year, v, ft, db)] == 0    

In [19]:
# 7. Yearly demand must be met for each distance bucket
for year in range(2023, 2039):
    for db in ['D1', 'D2', 'D3', 'D4']:
        for size in ['S1', 'S2', 'S3', 'S4']:
            demand = demand_df[(demand_df['Year'] == year) & (demand_df['Distance'] == db) & (demand_df['Size'] == size)]['Demand (km)'].values[0]
            prob += pl.lpSum([use_vars[(year, v, fuel, db)] * vehicles_df_add.loc[vehicles_df_add['ID'] == v , 'Yearly range (km)'].values[0]
                           for v in vehicles_df_add[(vehicles_df_add['Distance'] == db) & (vehicles_df_add['Size'] == size)]['ID']]) >= demand

In [28]:
# 8. Carbon emission limits must be respected
for year in range(2023, 2039):
    total_emissions = 0
    for _, row in merged_fuels_total_df.iterrows():
        for db in distance_bucket:
            # Unique vehicle id
            vehicle_id = row['ID']

            # Max distance travelled by vehicle, unit: km
            distance = row['Yearly range (km)']

            # Fuel type
            fuel_t = row['Fuel']
            
            # Carbon emissions, unit: (CO2/unit_fuel)
            emissions = row['Emissions (CO2/unit_fuel)']

            # Fuel consumption, unit: unit/km
            fuel_consumption = row['Consumption (unit_fuel/km)']

            # Calculate total emissions
            total_emissions += pl.lpSum([use_vars[(year, vehicle_id, fuel_t, db)] * fuel_consumption * distance * emissions])/4
    prob += total_emissions <= carbon_emissions_df[carbon_emissions_df['Year'] == year]['Carbon emission CO2/kg'].values[0]

69347.97596255998*Use_(2023,_'Diesel_S1_2023',_'B20',_'D1') + 69347.97596255998*Use_(2023,_'Diesel_S1_2023',_'B20',_'D2') + 69347.97596255998*Use_(2023,_'Diesel_S1_2023',_'B20',_'D3') + 69347.97596255998*Use_(2023,_'Diesel_S1_2023',_'B20',_'D4') + 11092.291306367999*Use_(2023,_'Diesel_S1_2023',_'HVO',_'D1') + 11092.291306367999*Use_(2023,_'Diesel_S1_2023',_'HVO',_'D2') + 11092.291306367999*Use_(2023,_'Diesel_S1_2023',_'HVO',_'D3') + 11092.291306367999*Use_(2023,_'Diesel_S1_2023',_'HVO',_'D4') + 69347.97596255998*Use_(2023,_'Diesel_S1_2024',_'B20',_'D1') + 69347.97596255998*Use_(2023,_'Diesel_S1_2024',_'B20',_'D2') + 69347.97596255998*Use_(2023,_'Diesel_S1_2024',_'B20',_'D3') + 69347.97596255998*Use_(2023,_'Diesel_S1_2024',_'B20',_'D4') + 11092.291306367999*Use_(2023,_'Diesel_S1_2024',_'HVO',_'D1') + 11092.291306367999*Use_(2023,_'Diesel_S1_2024',_'HVO',_'D2') + 11092.291306367999*Use_(2023,_'Diesel_S1_2024',_'HVO',_'D3') + 11092.291306367999*Use_(2023,_'Diesel_S1_2024',_'HVO',_'D4') + 

69347.97596255998*Use_(2025,_'Diesel_S1_2023',_'B20',_'D1') + 69347.97596255998*Use_(2025,_'Diesel_S1_2023',_'B20',_'D2') + 69347.97596255998*Use_(2025,_'Diesel_S1_2023',_'B20',_'D3') + 69347.97596255998*Use_(2025,_'Diesel_S1_2023',_'B20',_'D4') + 11092.291306367999*Use_(2025,_'Diesel_S1_2023',_'HVO',_'D1') + 11092.291306367999*Use_(2025,_'Diesel_S1_2023',_'HVO',_'D2') + 11092.291306367999*Use_(2025,_'Diesel_S1_2023',_'HVO',_'D3') + 11092.291306367999*Use_(2025,_'Diesel_S1_2023',_'HVO',_'D4') + 69347.97596255998*Use_(2025,_'Diesel_S1_2024',_'B20',_'D1') + 69347.97596255998*Use_(2025,_'Diesel_S1_2024',_'B20',_'D2') + 69347.97596255998*Use_(2025,_'Diesel_S1_2024',_'B20',_'D3') + 69347.97596255998*Use_(2025,_'Diesel_S1_2024',_'B20',_'D4') + 11092.291306367999*Use_(2025,_'Diesel_S1_2024',_'HVO',_'D1') + 11092.291306367999*Use_(2025,_'Diesel_S1_2024',_'HVO',_'D2') + 11092.291306367999*Use_(2025,_'Diesel_S1_2024',_'HVO',_'D3') + 11092.291306367999*Use_(2025,_'Diesel_S1_2024',_'HVO',_'D4') + 

69347.97596255998*Use_(2029,_'Diesel_S1_2023',_'B20',_'D1') + 69347.97596255998*Use_(2029,_'Diesel_S1_2023',_'B20',_'D2') + 69347.97596255998*Use_(2029,_'Diesel_S1_2023',_'B20',_'D3') + 69347.97596255998*Use_(2029,_'Diesel_S1_2023',_'B20',_'D4') + 11092.291306367999*Use_(2029,_'Diesel_S1_2023',_'HVO',_'D1') + 11092.291306367999*Use_(2029,_'Diesel_S1_2023',_'HVO',_'D2') + 11092.291306367999*Use_(2029,_'Diesel_S1_2023',_'HVO',_'D3') + 11092.291306367999*Use_(2029,_'Diesel_S1_2023',_'HVO',_'D4') + 69347.97596255998*Use_(2029,_'Diesel_S1_2024',_'B20',_'D1') + 69347.97596255998*Use_(2029,_'Diesel_S1_2024',_'B20',_'D2') + 69347.97596255998*Use_(2029,_'Diesel_S1_2024',_'B20',_'D3') + 69347.97596255998*Use_(2029,_'Diesel_S1_2024',_'B20',_'D4') + 11092.291306367999*Use_(2029,_'Diesel_S1_2024',_'HVO',_'D1') + 11092.291306367999*Use_(2029,_'Diesel_S1_2024',_'HVO',_'D2') + 11092.291306367999*Use_(2029,_'Diesel_S1_2024',_'HVO',_'D3') + 11092.291306367999*Use_(2029,_'Diesel_S1_2024',_'HVO',_'D4') + 

69347.97596255998*Use_(2031,_'Diesel_S1_2023',_'B20',_'D1') + 69347.97596255998*Use_(2031,_'Diesel_S1_2023',_'B20',_'D2') + 69347.97596255998*Use_(2031,_'Diesel_S1_2023',_'B20',_'D3') + 69347.97596255998*Use_(2031,_'Diesel_S1_2023',_'B20',_'D4') + 11092.291306367999*Use_(2031,_'Diesel_S1_2023',_'HVO',_'D1') + 11092.291306367999*Use_(2031,_'Diesel_S1_2023',_'HVO',_'D2') + 11092.291306367999*Use_(2031,_'Diesel_S1_2023',_'HVO',_'D3') + 11092.291306367999*Use_(2031,_'Diesel_S1_2023',_'HVO',_'D4') + 69347.97596255998*Use_(2031,_'Diesel_S1_2024',_'B20',_'D1') + 69347.97596255998*Use_(2031,_'Diesel_S1_2024',_'B20',_'D2') + 69347.97596255998*Use_(2031,_'Diesel_S1_2024',_'B20',_'D3') + 69347.97596255998*Use_(2031,_'Diesel_S1_2024',_'B20',_'D4') + 11092.291306367999*Use_(2031,_'Diesel_S1_2024',_'HVO',_'D1') + 11092.291306367999*Use_(2031,_'Diesel_S1_2024',_'HVO',_'D2') + 11092.291306367999*Use_(2031,_'Diesel_S1_2024',_'HVO',_'D3') + 11092.291306367999*Use_(2031,_'Diesel_S1_2024',_'HVO',_'D4') + 

69347.97596255998*Use_(2034,_'Diesel_S1_2023',_'B20',_'D1') + 69347.97596255998*Use_(2034,_'Diesel_S1_2023',_'B20',_'D2') + 69347.97596255998*Use_(2034,_'Diesel_S1_2023',_'B20',_'D3') + 69347.97596255998*Use_(2034,_'Diesel_S1_2023',_'B20',_'D4') + 11092.291306367999*Use_(2034,_'Diesel_S1_2023',_'HVO',_'D1') + 11092.291306367999*Use_(2034,_'Diesel_S1_2023',_'HVO',_'D2') + 11092.291306367999*Use_(2034,_'Diesel_S1_2023',_'HVO',_'D3') + 11092.291306367999*Use_(2034,_'Diesel_S1_2023',_'HVO',_'D4') + 69347.97596255998*Use_(2034,_'Diesel_S1_2024',_'B20',_'D1') + 69347.97596255998*Use_(2034,_'Diesel_S1_2024',_'B20',_'D2') + 69347.97596255998*Use_(2034,_'Diesel_S1_2024',_'B20',_'D3') + 69347.97596255998*Use_(2034,_'Diesel_S1_2024',_'B20',_'D4') + 11092.291306367999*Use_(2034,_'Diesel_S1_2024',_'HVO',_'D1') + 11092.291306367999*Use_(2034,_'Diesel_S1_2024',_'HVO',_'D2') + 11092.291306367999*Use_(2034,_'Diesel_S1_2024',_'HVO',_'D3') + 11092.291306367999*Use_(2034,_'Diesel_S1_2024',_'HVO',_'D4') + 

69347.97596255998*Use_(2036,_'Diesel_S1_2023',_'B20',_'D1') + 69347.97596255998*Use_(2036,_'Diesel_S1_2023',_'B20',_'D2') + 69347.97596255998*Use_(2036,_'Diesel_S1_2023',_'B20',_'D3') + 69347.97596255998*Use_(2036,_'Diesel_S1_2023',_'B20',_'D4') + 11092.291306367999*Use_(2036,_'Diesel_S1_2023',_'HVO',_'D1') + 11092.291306367999*Use_(2036,_'Diesel_S1_2023',_'HVO',_'D2') + 11092.291306367999*Use_(2036,_'Diesel_S1_2023',_'HVO',_'D3') + 11092.291306367999*Use_(2036,_'Diesel_S1_2023',_'HVO',_'D4') + 69347.97596255998*Use_(2036,_'Diesel_S1_2024',_'B20',_'D1') + 69347.97596255998*Use_(2036,_'Diesel_S1_2024',_'B20',_'D2') + 69347.97596255998*Use_(2036,_'Diesel_S1_2024',_'B20',_'D3') + 69347.97596255998*Use_(2036,_'Diesel_S1_2024',_'B20',_'D4') + 11092.291306367999*Use_(2036,_'Diesel_S1_2024',_'HVO',_'D1') + 11092.291306367999*Use_(2036,_'Diesel_S1_2024',_'HVO',_'D2') + 11092.291306367999*Use_(2036,_'Diesel_S1_2024',_'HVO',_'D3') + 11092.291306367999*Use_(2036,_'Diesel_S1_2024',_'HVO',_'D4') + 

69347.97596255998*Use_(2038,_'Diesel_S1_2023',_'B20',_'D1') + 69347.97596255998*Use_(2038,_'Diesel_S1_2023',_'B20',_'D2') + 69347.97596255998*Use_(2038,_'Diesel_S1_2023',_'B20',_'D3') + 69347.97596255998*Use_(2038,_'Diesel_S1_2023',_'B20',_'D4') + 11092.291306367999*Use_(2038,_'Diesel_S1_2023',_'HVO',_'D1') + 11092.291306367999*Use_(2038,_'Diesel_S1_2023',_'HVO',_'D2') + 11092.291306367999*Use_(2038,_'Diesel_S1_2023',_'HVO',_'D3') + 11092.291306367999*Use_(2038,_'Diesel_S1_2023',_'HVO',_'D4') + 69347.97596255998*Use_(2038,_'Diesel_S1_2024',_'B20',_'D1') + 69347.97596255998*Use_(2038,_'Diesel_S1_2024',_'B20',_'D2') + 69347.97596255998*Use_(2038,_'Diesel_S1_2024',_'B20',_'D3') + 69347.97596255998*Use_(2038,_'Diesel_S1_2024',_'B20',_'D4') + 11092.291306367999*Use_(2038,_'Diesel_S1_2024',_'HVO',_'D1') + 11092.291306367999*Use_(2038,_'Diesel_S1_2024',_'HVO',_'D2') + 11092.291306367999*Use_(2038,_'Diesel_S1_2024',_'HVO',_'D3') + 11092.291306367999*Use_(2038,_'Diesel_S1_2024',_'HVO',_'D4') + 

In [ ]:
#carbon_emissions_df[carbon_emissions_df['Year'] == 2038]['Carbon emission CO2/kg'].values[0]

In [30]:
merged_fuels_total_df

,ID,Fuel,Consumption (unit_fuel/km),Vehicle,Size,Year,Cost ($),Yearly range (km),Distance,Emissions (CO2/unit_fuel),Cost ($/unit_fuel),Cost Uncertainty (±%)
0,BEV_S1_2023,Electricity,0.893043,BEV,S1,2023,187000,102000,D1,0.000000,0.191791,0
1,BEV_S2_2023,Electricity,0.896800,BEV,S2,2023,272000,106000,D1,0.000000,0.191791,0
2,BEV_S4_2023,Electricity,0.900664,BEV,S4,2023,395636,118000,D1,0.000000,0.191791,0
3,BEV_S3_2023,Electricity,0.898003,BEV,S3,2023,309090,73000,D1,0.000000,0.191791,0
4,BEV_S1_2024,Electricity,0.893043,BEV,S1,2024,177650,102000,D1,0.000000,0.184113,2
5,BEV_S2_2024,Electricity,0.896800,BEV,S2,2024,258400,106000,D1,0.000000,0.184113,2
6,BEV_S4_2024,Electricity,0.900664,BEV,S4,2024,375854,118000,D1,0.000000,0.184113,2
7,BEV_S3_2024,Electricity,0.898003,BEV,S3,2024,293636,73000,D1,0.000000,0.184113,2
8,BEV_S1_2025,Electricity,0.893043,BEV,S1,2025,168767,102000,D1,0.000000,0.176742,4
9,BEV_S2_2025,Electricity,0.896800,BEV,S2,2025,245480,106000,D1,0.000000,0.176742,4


***

In [25]:
# Define solver parameters so that won't run infinitely
solver = pl.PULP_CBC_CMD(
    timeLimit=60,         # Maximum time in seconds
    gapRel=0.01,           # Relative gap tolerance       
    threads=4,              # Number of threads to use
    msg=True                # Display solver messages
)

In [26]:
prob.solve(solver)

-1

In [22]:
for y in range(2023, 2039):
    for v in vehicles_df['ID']:
        if pl.value(buy_vars[(y, v)]) >0:
            print(y,v, pl.value(buy_vars[(y, v)]))

2023 Diesel_S1_2023 233.0
2023 Diesel_S2_2023 76.0
2023 Diesel_S4_2023 24.0
2023 Diesel_S3_2023 181.0
2024 Diesel_S1_2024 244.0
2024 Diesel_S2_2024 76.0
2024 Diesel_S4_2024 24.0
2024 Diesel_S3_2024 185.0
2025 Diesel_S1_2025 195.0
2025 Diesel_S2_2025 78.0
2025 Diesel_S4_2025 25.0
2025 Diesel_S3_2025 189.0
2026 Diesel_S1_2026 156.0
2026 Diesel_S2_2026 79.0
2026 Diesel_S4_2026 66.0
2026 Diesel_S3_2026 206.0
2027 Diesel_S1_2027 125.0
2027 Diesel_S2_2027 82.0
2027 Diesel_S4_2027 12.0
2027 Diesel_S3_2027 286.0
2028 Diesel_S1_2028 131.0
2028 Diesel_S2_2028 133.0
2028 Diesel_S4_2028 12.0
2028 Diesel_S3_2028 220.0
2029 Diesel_S1_2029 86.0
2029 Diesel_S2_2029 53.0
2029 Diesel_S4_2029 36.0
2029 Diesel_S3_2029 286.0
2030 Diesel_S1_2030 260.0
2030 Diesel_S2_2030 65.0


In [23]:
for y in range(2023, 2039):
    for v in vehicles_df['ID']:
        for d in distance_bucket:
            for f in fuel_type:
                if pl.value(use_vars[(y, v, f, d)]) >0:
                    print((y,v,f, d, pl.value(use_vars[(y, v, f, d)]) ))

(2023, 'Diesel_S1_2023', 'HVO', 'D1', 9.0)
(2023, 'Diesel_S1_2023', 'HVO', 'D2', 26.0)
(2023, 'Diesel_S1_2023', 'HVO', 'D3', 33.0)
(2023, 'Diesel_S1_2023', 'HVO', 'D4', 5.0)
(2023, 'Diesel_S2_2023', 'HVO', 'D1', 10.0)
(2023, 'Diesel_S2_2023', 'HVO', 'D2', 14.0)
(2023, 'Diesel_S2_2023', 'HVO', 'D3', 8.0)
(2023, 'Diesel_S2_2023', 'HVO', 'D4', 2.0)
(2023, 'Diesel_S4_2023', 'HVO', 'D1', 1.0)
(2023, 'Diesel_S4_2023', 'HVO', 'D2', 7.0)
(2023, 'Diesel_S4_2023', 'HVO', 'D3', 2.0)
(2023, 'Diesel_S4_2023', 'HVO', 'D4', 1.0)
(2023, 'Diesel_S3_2023', 'HVO', 'D1', 30.0)
(2023, 'Diesel_S3_2023', 'HVO', 'D2', 34.0)
(2023, 'Diesel_S3_2023', 'HVO', 'D3', 14.0)
(2023, 'Diesel_S3_2023', 'HVO', 'D4', 3.0)
(2024, 'Diesel_S1_2024', 'HVO', 'D1', 9.0)
(2024, 'Diesel_S1_2024', 'HVO', 'D2', 27.0)
(2024, 'Diesel_S1_2024', 'HVO', 'D3', 33.0)
(2024, 'Diesel_S1_2024', 'HVO', 'D4', 5.0)
(2024, 'Diesel_S2_2024', 'HVO', 'D1', 10.0)
(2024, 'Diesel_S2_2024', 'HVO', 'D2', 14.0)
(2024, 'Diesel_S2_2024', 'HVO', 'D3', 8.0)


In [24]:
#for v in vehicles_df['ID']:
#    for d in distance_bucket:
#        for f in fuel_type:
#            if pl.value(use_vars[(y, v, f, d)]) >0 & (y == 2038):
#                print((y,v,f, d, pl.value(use_vars[(y, v, f, d)]) ))

In [25]:
for y in range(2023, 2039):
    for v in vehicles_df['ID']:
        if pl.value(sell_vars[(y, v)]) is not None:
            if pl.value(sell_vars[(y, v)]) >0:
                print(y, v, pl.value(sell_vars[(y, v)]))

2024 Diesel_S2_2023 3.0
2024 Diesel_S4_2023 24.0
2024 Diesel_S3_2023 181.0
2025 Diesel_S2_2024 55.0
2025 Diesel_S4_2024 24.0
2025 Diesel_S3_2024 185.0
2026 Diesel_S2_2024 21.0
2026 Diesel_S2_2025 78.0
2026 Diesel_S4_2025 25.0
2026 Diesel_S3_2025 189.0
2027 Diesel_S2_2026 79.0
2027 Diesel_S4_2026 66.0
2027 Diesel_S3_2026 206.0
2028 Diesel_S2_2027 82.0
2028 Diesel_S4_2027 12.0
2028 Diesel_S3_2027 286.0
2029 Diesel_S1_2028 31.0
2029 Diesel_S2_2028 133.0
2029 Diesel_S4_2028 12.0
2029 Diesel_S3_2028 220.0
2030 Diesel_S1_2029 6.0
2030 Diesel_S2_2029 53.0
2030 Diesel_S4_2029 36.0
2030 Diesel_S3_2029 286.0
2031 Diesel_S1_2023 233.0
2031 Diesel_S2_2023 73.0
2032 Diesel_S1_2024 244.0
2033 Diesel_S1_2025 195.0
2034 Diesel_S1_2026 156.0
2035 Diesel_S1_2027 125.0
2036 Diesel_S1_2028 100.0
2037 Diesel_S1_2029 80.0
2038 Diesel_S2_2030 64.0


In [26]:
output = []
for year in range(2023, 2039):
    for vehicle_id in vehicles_df['ID']:
        if pl.value(buy_vars[(year, vehicle_id)]) > 0:
            output.append([
                year,
                vehicle_id,
                pl.value(buy_vars[(year, vehicle_id)]),
                'Buy',
                'NaN',
                'NaN',
                0.0
            ])
        if pl.value(use_vars[(year, vehicle_id, fuel, db)]) > 0:
            for db in ['D1', 'D2', 'D3', 'D4']:
                if pl.value(distance_bucket_choice_vars[(year, vehicle_id, distance_bucket)]) > 0:
                    for fuel in ['Electricity', 'B20', 'LNG', 'BioLNG', 'HVO']:
                        if pl.value(fuel_choice_vars[(year, vehicle_id, fuel)]) > 0:
                            output.append([
                                year,
                                vehicle_id,
                                pl.value(use_vars[(year, vehicle_id)]),
                                'Use',
                                fuel,
                                distance_bucket,
                                pl.value(distance_vars[(year, vehicle_id)])
                            ])
        if pl.value(sell_vars[(year, vehicle_id)]) > 0:
            output.append([
                year,
                vehicle_id,
                pl.value(sell_vars[(year, vehicle_id)]),
                'Sell',
                'NaN',
                'NaN',
                0.0
            ])

output_df = pd.DataFrame(output, columns=[
    'Year', 'ID', 'Num_Vehicles', 'Type', 'Fuel', 'Distance_bucket', 'Distance_per_vehicle(km)'
])

TypeError: '>' not supported between instances of 'NoneType' and 'int'